In [1]:
#Kraabime veebilehelt PDFid ja slaidiesitlused maha

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
#Tegin siia meetodi, mis võtab nasdaqbalticu lehelt siis ISIN koodi ja kuupäeva järgi kõik .pdf formaadis failid,
#laeb need lokaalselt mulle alla.
def fetch_pdfs(ISIN,date):
    url = f"https://nasdaqbaltic.com/statistics/en/instrument/{ISIN}/reports?date={date}"
    #If there is no such folder, the script will create one automatically
    folder_location = r'C:\Users\marek.keskull\webscraping'
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    tr = soup.find("tbody")
    pdf2020 = tr.find("tr")
    
    for link in pdf2020.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)
fetch_pdfs("EE3100007857","2021-01-11")

In [2]:
#kasutan tika libraryt, selle peab panema lokaalselt käima eraldi, et kasutada tika rest teenust.
import tika
from tika import parser

import os

folder_location = r'C:\Users\marek.keskull\webscraping'
#Võtan kõik allalaetud PDF'id ning muudan need .txt failideks, et mul jääks alles ainult plain text
def extract_text_from_pdfs_recursively(dir):
    for root, dirs, files in os.walk(dir):
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            if ext == '.pdf':
                print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)
                path_to_txt = stem + '.txt'
                with open(path_to_txt, 'w',encoding="utf-8") as txt_file:
                    print("Writing contents to " + path_to_txt)
                    txt_file.write(pdf_contents['content'])
                    


if __name__ == "__main__":
    extract_text_from_pdfs_recursively(folder_location)

Processing C:\Users\marek.keskull\webscraping\2020_q1_en_eur_00_00.pdf


2021-01-24 19:32:08,836 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Writing contents to C:\Users\marek.keskull\webscraping\2020_q1_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q1_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q1_en_eur_con_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q2_en_eur_00_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q2_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q2_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q2_en_eur_con_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q3_en_eur_00_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q3_en_eur_00_00.txt
Processing C:\Users\marek.keskull\webscraping\2020_q3_en_eur_con_00.pdf
Writing contents to C:\Users\marek.keskull\webscraping\2020_q3_en_eur_con_00.txt


In [3]:
quarters = ['2020_q1_en_eur_00_00','2020_q1_en_eur_con_00','2020_q2_en_eur_00_00','2020_q2_en_eur_con_00',
            '2020_q3_en_eur_00_00','2020_q3_en_eur_con_00']
import re
pattern = re.compile(r'\s+')

data = []
for c in quarters:
    with open(folder_location+ "\\" + c + ".txt", "rb") as file:
        text = file.read()
        asd = text.strip()
        decoded=str(asd,'utf-8')
        #cleantext = re.sub(pattern, '', text)
        data.append(decoded)
data

["PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of \r\n\r\n21.04.2020\r\n\r\n\r\n\r\nResults of Q1 2020 – compared to Q4 2019\r\n\r\nKey indicators in quarterly comparison • Revenues +5%\r\n\r\n• Net interest increased +0.5m€ as loan portfolio\r\n\r\ngrew by 44m€\r\n\r\n• Net service fees 0.15m€ lower due to certain\r\n\r\none-off service fees in Q4 2019\r\n\r\n• Operating costs -4%\r\n\r\n• Operating profit +26%\r\n\r\n• Loan impairment losses +0.7m€ of which\r\n\r\n• +0.4m€ additional impairment in relation to\r\n\r\npayment holidays arising from national emergency\r\n\r\nsituation\r\n\r\n• Net profit -8% (impacted by loan impairments)\r\n\r\nQ1 2020 Q4 2019 Quarterly change\r\n\r\nNet operating income ('000 €) 7 117 6 756 +361 +5%\r\n\r\nInterest 6 358 5 819 +539 +9%\r\n\r\nService fee and commissions 521 675 -154 -23%\r\n\r\nOther 238 262 -24 -9%\r\n\r\nOperating expenses -4 442 -4 635 +193 -4%\r\n\r\nOperating profit 2 675 2 121 +554 +26%\r\n\r\nLoan 

In [4]:
#teen listidest dictionary
ddata = dict(zip(quarters, data))
ddata

{'2020_q1_en_eur_00_00': "PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of \r\n\r\n21.04.2020\r\n\r\n\r\n\r\nResults of Q1 2020 – compared to Q4 2019\r\n\r\nKey indicators in quarterly comparison • Revenues +5%\r\n\r\n• Net interest increased +0.5m€ as loan portfolio\r\n\r\ngrew by 44m€\r\n\r\n• Net service fees 0.15m€ lower due to certain\r\n\r\none-off service fees in Q4 2019\r\n\r\n• Operating costs -4%\r\n\r\n• Operating profit +26%\r\n\r\n• Loan impairment losses +0.7m€ of which\r\n\r\n• +0.4m€ additional impairment in relation to\r\n\r\npayment holidays arising from national emergency\r\n\r\nsituation\r\n\r\n• Net profit -8% (impacted by loan impairments)\r\n\r\nQ1 2020 Q4 2019 Quarterly change\r\n\r\nNet operating income ('000 €) 7 117 6 756 +361 +5%\r\n\r\nInterest 6 358 5 819 +539 +9%\r\n\r\nService fee and commissions 521 675 -154 -23%\r\n\r\nOther 238 262 -24 -9%\r\n\r\nOperating expenses -4 442 -4 635 +193 -4%\r\n\r\nOperating profit 2 675 2 12

In [5]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [6]:
data_combined = {key: [combine_text(value)] for (key, value) in ddata.items()}

In [7]:
#pandas dataframe data
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['text']
data_df = data_df.sort_index()
data_df

,text
2020_q1_en_eur_00_00,PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of \r\n\r\n21.04.2020\r\n\r\n\r\n\r\nResults of Q1 2020 – compared to ...
2020_q1_en_eur_con_00,• \r\n\r\n• \r\n\r\n•
2020_q2_en_eur_00_00,PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of \r\n\r\n21.07.2020\r\n\r\n\r\n\r\nDeposits\r\n\r\nOperating and net...
2020_q2_en_eur_con_00,0\r\n\r\n100\r\n\r\n200\r\n\r\n300\r\n\r\n400\r\n\r\n500\r\n\r\n600\r\n\r\n700\r\n\r\n800\r\n\r\n30.06.2019 30.09.2019 31.12.2019 31.03.2020 30.06...
2020_q3_en_eur_00_00,PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of Q3 2020\r\n\r\n21.10.2020\r\n\r\n\r\n\r\nKey points of Q3 2020\r\n\...
2020_q3_en_eur_con_00,Microsoft Word - Interim report_9m 2020_EN.docx\r\n\r\n\r\n1 \r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n ...


In [91]:
#vaatan datat 
data_df.text.loc['2020_q3_en_eur_00_00']


"PowerPoint Presentation\r\n\r\n\r\nCoop Pank Group\r\n\r\nUnaudited results of Q3 2020\r\n\r\n21.10.2020\r\n\r\n\r\n\r\nKey points of Q3 2020\r\n\r\n• Continued growth and stable quality of loan portfolio\r\n\r\n• Inflow of domestic deposits has increased significantly\r\n\r\n• Key performance indicators are improving (cost / income ratio, ROE)\r\n\r\n\r\n\r\nDeposits\r\n\r\nOperating and net profitDaily banking clients\r\n\r\nBusiness volumes\r\n\r\nLoans\r\n\r\nChange reflects difference in net profit\r\n\r\n418  460  \r\n504  559  \r\n\r\n599  \r\n\r\nQ3 2019 Q4 Q1 2020 Q2 Q3\r\n\r\nm€ +181 +43%\r\n\r\n+40 +7%\r\n\r\n478  507  \r\n547  \r\n\r\n615  \r\n688  \r\n\r\nQ3 2019 Q4 Q1 2020 Q2 Q3\r\n\r\nm€ +210 +44%\r\n\r\n+73 +12%\r\n\r\n56  64  \r\n69  75  \r\n\r\n80  \r\n\r\nQ3 2019 Q4 Q1 2020 Q2 Q3\r\n\r\n'000 +24 +42%\r\n\r\n+5 +7%\r\n\r\n2 091  2 121  \r\n2 675  \r\n\r\n3 109  3 262  \r\n\r\n1 519  1 583  1 461  1 362  \r\n2 012  \r\n\r\nQ3 2019 Q4 Q1 2020 Q2 Q3\r\n\r\nOperating pro

# Andmete puhastamine


- Tekst kõik väikeste tähtedega
- Eemaldame punktid
- Eemaldame numbrid
- Eemdaldame ebavajaliku teksti, mis on kandiliste sulgude vahel


In [95]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

round1 = lambda x: clean_text_round1(x)

In [96]:
#vaatame puhastatud teksti
data_clean = pd.DataFrame(data_df.text.apply(round1))
data_clean

,text
2020_q1_en_eur_00_00,powerpoint presentation\r\n\r\n\r\ncoop pank group\r\n\r\nunaudited results of \r\n\r\n \r\n\r\n\r\n\r\nresults of – compared to \r\n\r\nkey i...
2020_q1_en_eur_con_00,• \r\n\r\n• \r\n\r\n•
2020_q2_en_eur_00_00,powerpoint presentation\r\n\r\n\r\ncoop pank group\r\n\r\nunaudited results of \r\n\r\n \r\n\r\n\r\n\r\ndeposits\r\n\r\noperating and net profitd...
2020_q2_en_eur_con_00,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n \r\n\r\nfinancial assets\r\n\r\nprivate loans corporate loan...
2020_q3_en_eur_00_00,powerpoint presentation\r\n\r\n\r\ncoop pank group\r\n\r\nunaudited results of \r\n\r\n \r\n\r\n\r\n\r\nkey points of \r\n\r\n• continued growt...
2020_q3_en_eur_con_00,microsoft word interim report en docx\r\n\r\n\r\n \r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r...


In [97]:
data_clean.text.loc['2020_q3_en_eur_00_00']

'powerpoint presentation\r\n\r\n\r\ncoop pank group\r\n\r\nunaudited results of  \r\n\r\n  \r\n\r\n\r\n\r\nkey points of  \r\n\r\n• continued growth and stable quality of loan portfolio\r\n\r\n• inflow of domestic deposits has increased significantly\r\n\r\n• key performance indicators are improving  cost   income ratio  roe \r\n\r\n\r\n\r\ndeposits\r\n\r\noperating and net profitdaily banking clients\r\n\r\nbusiness volumes\r\n\r\nloans\r\n\r\nchange reflects difference in net profit\r\n\r\n    \r\n    \r\n\r\n  \r\n\r\n      \r\n\r\nm€     \r\n\r\n    \r\n\r\n    \r\n  \r\n\r\n  \r\n  \r\n\r\n      \r\n\r\nm€     \r\n\r\n    \r\n\r\n    \r\n    \r\n\r\n  \r\n\r\n      \r\n\r\n      \r\n\r\n    \r\n\r\n      \r\n   \r\n\r\n      \r\n\r\n            \r\n   \r\n\r\n      \r\n\r\noperating profit net profit\r\n\r\n  €     \r\n\r\n    \r\n\r\n\r\n\r\nresults of   – compared to  \r\n\r\nkey indicators in quarterly comparison • revenues   \r\n\r\n• net interest increased   € as loan portfol

In [101]:
# 2. round puhastamisele
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    #text = re.sub('https?://\S+|www\.\S+', '', text)
    #text = re.sub('<.*?>+', '', text)
    text = re.sub('\r', ' ', text)
    #text = re.sub(r"\s+"," ", text)
    #text = re.sub(r"^\s+", "", text)
    #text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    #text = re.sub(r'/\d\.\s+|[a-z]\)\s+|•\s+|[A-Z]\.\s+|[IVX]+\.\s+/g', "",text)
    #text = re.sub(r'\xc2\xb7',"",text)
    text = re.sub(r'\W+', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [102]:
#vaatame puhastatud teksti
data_clean = pd.DataFrame(data_clean.text.apply(round2))
data_clean

,text
2020_q1_en_eur_00_00,powerpoint presentation coop pank group unaudited results of results of compared to key indicators in quarterly comparison revenues net interest i...
2020_q1_en_eur_con_00,
2020_q2_en_eur_00_00,powerpoint presentation coop pank group unaudited results of deposits operating and net profitdaily banking clients business volumes loans change ...
2020_q2_en_eur_con_00,financial assets private loans corporate loans cash and bonds mln eur customer deposits and loans received private individuals corporates mln eur
2020_q3_en_eur_00_00,powerpoint presentation coop pank group unaudited results of key points of continued growth and stable quality of loan portfolio inflow of domesti...
2020_q3_en_eur_con_00,microsoft word interim report en docx interim report months table of contents management report overview of financial results capitalisation conso...


In [113]:
data_clean.text.loc['2020_q1_en_eur_con_00']

' '

In [108]:
#moodustame document term matrix

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,accordance,according,account,accounted,accounting,accounts,accumulated,acquisition,activities,activity,...,volume,volumes,weighted,word,www,year,years,ühistukapital,šiaulių,škodas
2020_q1_en_eur_00_00,0,0,1,0,0,0,0,0,0,0,...,2,1,0,0,1,3,0,1,1,0
2020_q1_en_eur_con_00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020_q2_en_eur_00_00,0,0,1,0,0,0,0,0,0,1,...,2,1,0,0,1,4,0,1,1,0
2020_q2_en_eur_con_00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020_q3_en_eur_00_00,0,0,2,0,0,0,0,0,0,0,...,2,1,0,0,1,4,0,1,1,0
2020_q3_en_eur_con_00,1,5,4,1,5,1,1,1,10,1,...,3,1,6,1,1,5,4,0,0,1


In [109]:
data = data_dtm.transpose()
data.head()

,2020_q1_en_eur_00_00,2020_q1_en_eur_con_00,2020_q2_en_eur_00_00,2020_q2_en_eur_con_00,2020_q3_en_eur_00_00,2020_q3_en_eur_con_00
accordance,0,0,0,0,0,1
according,0,0,0,0,0,5
account,1,0,1,0,2,4
accounted,0,0,0,0,0,1
accounting,0,0,0,0,0,5


In [110]:
# top 30 sõna sõnastikus
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{'2020_q1_en_eur_00_00': [('loan', 21),
  ('portfolio', 20),
  ('loans', 19),
  ('net', 19),
  ('share', 16),
  ('profit', 15),
  ('deposits', 14),
  ('operating', 12),
  ('coop', 11),
  ('pank', 10),
  ('business', 10),
  ('gross', 10),
  ('clients', 9),
  ('market', 9),
  ('cost', 9),
  ('increased', 8),
  ('capital', 8),
  ('income', 8),
  ('ratio', 7),
  ('total', 7),
  ('past', 7),
  ('private', 7),
  ('loss', 6),
  ('allowances', 6),
  ('shareholders', 6),
  ('growth', 5),
  ('service', 5),
  ('investors', 4),
  ('price', 4),
  ('costs', 4)],
 '2020_q1_en_eur_con_00': [('škodas', 0),
  ('exit', 0),
  ('everyday', 0),
  ('events', 0),
  ('evaluation', 0),
  ('euros', 0),
  ('european', 0),
  ('eur', 0),
  ('eu', 0),
  ('estonian', 0),
  ('estonia', 0),
  ('estate', 0),
  ('equivalents', 0),
  ('equity', 0),
  ('equipment', 0),
  ('equal', 0),
  ('entities', 0),
  ('end', 0),
  ('en', 0),
  ('employees', 0),
  ('emor', 0),
  ('exchange', 0),
  ('expense', 0),
  ('elimination', 0),


In [111]:
import pickle
data_dtm.to_pickle("dtm.pkl")

In [112]:
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))